In [1]:
import sys
sys.path.append("../")

import pandas as pd

In [5]:
from config_utils.load_config import load_params_from_yaml, DataParamsSchema
dataset_params = load_params_from_yaml("dataset_params.yaml", DataParamsSchema)

from pathlib import Path
root_dir = Path().resolve().parents[0]

In [14]:
train_df = pd.read_csv(root_dir / dataset_params.data_params.train_data_path)
test_df = pd.read_csv(root_dir / dataset_params.data_params.test_data_path)
subred_1 = pd.read_csv(root_dir / dataset_params.data_params.subset1_path /  (dataset_params.data_params.subreddit1 + '.csv'))
subred_2 = pd.read_csv(root_dir / dataset_params.data_params.subset2_path /  (dataset_params.data_params.subreddit2 + '.csv'))


In [15]:
train_df.head()

,title,body,subreddit
0,DAE heart go crazy at random times? I'm not su...,Just 10 minutes ago I freaked out because my h...,Health
1,fixing the wrong break-up?,So I had been dating this girl for about a yea...,relationship_advice
2,Another existential crisis about higher educat...,Everyone is probably getting tired of these by...,TwoXChromosomes
3,What is the best bank for a savings account?,I'm trying to decide what the best bank is to ...,AskReddit
4,(Very) Beginning leg workout for the out of shape,While doing my rehab exercises for a bad hammy...,Fitness


In [16]:
test_df.head()


,title,body,subreddit
0,[8] i wish i could have baggies evrywhere,i like redorange envelops but i'd like baggies...,trees
1,The Movie or Movies that scared the shit out o...,Three for me. 1. Pinocchio. Don't know why but...,movies
2,Question regarding bringing back extinct speci...,This is probably gonna be a stupid post as I k...,biology
3,Anyone here into crossdressing?,I've been crossdressing for a while now and I'...,gonewild
4,"Can anyone help me identify this plant, please?",Here are a couple of pictures. This plant is ...,answers


In [17]:
subred_1.head()

,title,body,subreddit
0,"Programmers, do you put your comments (before|...",Inspired by: \n\nIn the several years I've bee...,programming
1,Anyone want to give some good 'finding a job' ...,I'm approaching the end of my junior year of c...,programming
2,I cranked out an awesome game in 48 hours for ...,The global game jam went down this past weeken...,programming
3,The best way to store passwords in your applic...,...is only as good as the policy you enforce. ...,programming
4,"Hey Proggit here is my first program, a mixtur...",It's an application that emails you when your ...,programming


In [18]:
subred_2.head()

,title,body,subreddit
0,Here's an idea: Left 4 Dead Jurassic Park,After the power to the park goes out and the d...,gaming
1,"r/gaming, I'm looking for a good quality heads...",I've been using this crappy little headset tha...,gaming
2,Please help me figure out why it's so tough fo...,"4 times out of 5, when I try to connect to a V...",gaming
3,How to rob your opponent of a kill in MW2 in l...,Its quite easy. All you have to do is drop a g...,gaming
4,Anybody else find it hard to open a game after...,"I don't know if I'm weird or what, but every t...",gaming


In [19]:
len(train_df), len(test_df), len(subred_1), len(subred_2)

(60000, 6000, 465, 1000)